In [11]:
# Add RMG-Py and ARC to Python Path (Optional)
import sys
# this is an example path if running the notebook on supercloud
# replace <USERNAME> with your username
# sys.path.insert(0, "/home/gridsan/hwpang/Software/RMG-Py/")

import os
import glob
import shutil

import numpy as np
import subprocess

import ase.atoms

import arkane.encorr.reference
import arkane.ess
import rmgpy.molecule
# from arkane.encorr.reference import ReferenceDatabase
# from arkane.ess import ess_factory
# from rmgpy.molecule import Molecule

# import sys
# sys.path.insert(0, "/home/gridsan/hwpang/Software/RDMC/")

In [4]:
# load the reference database
database = arkane.encorr.reference.ReferenceDatabase()
database.load()

*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3);
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3);
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual v

*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(2)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(2)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(1)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(1)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): O(1)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): O(1)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): O+1(2)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): O+1(2)
ERROR:root:Unable to generate identifier for this molecule:
1 O u0 p3 c-1 {3,S}
2 O u0 p2 c0 {3,D}
3 C u0 p0 c0 {1,S} {2,D} {4,S}
4 H u0 p0 c0 {3,S}

*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valen

*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): S(1)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): S(1)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): S(1); O(1)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): S(1); O(1)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): O(1)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): O(1)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): O(1)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): O(1)
*** Open Babel Warning  in InChI code
  #1 :Accepted

In [5]:
# working_dir = '/home/harris.se/rmg/rmg_tools/arkane/new_lot/bac_calcs/ccsdt_ccpvtz'
working_dir = '/scratch/harris.se/guassian_scratch/bac'

In [4]:
arkane.ess.ess_factory

<function arkane.ess.factory.ess_factory(fullpath: str, check_for_errors: bool = True, scratch_directory: str = None) -> Type[arkane.ess.adapter.ESSAdapter]>

In [6]:
def get_xyz(atoms, fmt='%22.15f'):  # taken from ase.io.xyz's write_xyz function
    # returns the positions of the ase.atoms.Atoms object in xyz format as a string
    xyz_str = ""
    for s, (x, y, z) in zip(atoms.symbols, atoms.positions):
        xyz_str += ('%-2s %s %s %s\n' % (s, fmt % x, fmt % y, fmt % z))
    return xyz_str

In [6]:
# for i in range(len(database.reference_sets['main'])):
for i in incomplete:
#     try:
#         sp = database.get_species_from_index(i)
#     except ValueError:
#         continue
        
    sp_dir = os.path.join(working_dir, f'species_{i:04}')
    
    gaussian_log = os.path.join(sp_dir, 'sp.log')
    # make an orca file
    
    try:
        my_log = arkane.ess.ess_factory(gaussian_log)
    except arkane.exceptions.LogError:
        continue
    # make a run file
    coord, number, mass = my_log.load_geometry()
    my_atoms = ase.Atoms(number, coord)
    
    
    parallel = True
    orca_input_file = os.path.join(sp_dir, 'conformer.inp')
    input_format = """!{res}HF {level_of_theory} TightSCF tightPNO
!energy

%maxcore 7000
{opt_parallel_line}

* xyz {charge} {mult}
{xyz}*


"""
    charge = database.reference_sets['main'][i].charge
    multiplicity = database.reference_sets['main'][i].multiplicity
    input_content = input_format.format(
        res='r' if multiplicity == 1 else 'u',
        level_of_theory='dlpno-ccsd(t)-f12 cc-pvtz-f12 aug-cc-pvtz/c cc-pvtz-f12-cabs',
        opt_parallel_line='%pal nprocs 16 end' if parallel else '',
        charge=charge,
        mult=multiplicity,
        xyz=get_xyz(my_atoms),
    )
    
    with open(orca_input_file, 'w') as f:
        f.write(input_content)
        
    run_orca_script = os.path.join(sp_dir, 'run_orca.sh')
    with open(run_orca_script, 'w') as f:
        # TODO format the text without """ so it doesn't mess with VSCode's collapse function button
        f.write("""#!/bin/bash
#SBATCH --job-name=""" + f'orca_{i:04}' + """
#SBATCH --error=error.log
#SBATCH --nodes=1
#SBATCH --partition=west,short
#SBATCH --exclude=c5003
#SBATCH --mem=200Gb
#SBATCH --time=24:00:00
#SBATCH --ntasks=16


ompi=/work/westgroup/orca/openmpi-4.1.6/build
PATH=$ompi/bin:$PATH
LD_LIBRARY_PATH=$ompi/lib:$ompi/etc:$LD_LIBRARY_PATH

#Orca
orcadir=/work/westgroup/orca/orca_6_0_1_linux_x86-64_shared_openmpi416
export PATH=$PATH:$orcadir
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$orcadir

$orcadir/orca conformer.inp > conformer.out
""")

In [16]:
incomplete = []
for i in range(len(database.reference_sets['main'])):
    sp_dir = os.path.join(working_dir, f'species_{i:04}')
    ol_file = os.path.join(sp_dir, 'conformer.out')
    if not os.path.exists(ol_file):
        print(f'sbatch run {sp_dir}/run_orca.sh')
        incomplete.append(i)
        continue
    try:
        o_ess = arkane.ess.ess_factory(ol_file)
    except arkane.exceptions.LogError:
        print(f'sbatch run {sp_dir}/run_orca.sh')
        incomplete.append(i)

sbatch run /scratch/harris.se/guassian_scratch/bac/species_0002/run_orca.sh
sbatch run /scratch/harris.se/guassian_scratch/bac/species_0004/run_orca.sh
sbatch run /scratch/harris.se/guassian_scratch/bac/species_0005/run_orca.sh
sbatch run /scratch/harris.se/guassian_scratch/bac/species_0006/run_orca.sh
sbatch run /scratch/harris.se/guassian_scratch/bac/species_0007/run_orca.sh
sbatch run /scratch/harris.se/guassian_scratch/bac/species_0008/run_orca.sh
sbatch run /scratch/harris.se/guassian_scratch/bac/species_0009/run_orca.sh
sbatch run /scratch/harris.se/guassian_scratch/bac/species_0010/run_orca.sh
sbatch run /scratch/harris.se/guassian_scratch/bac/species_0011/run_orca.sh
sbatch run /scratch/harris.se/guassian_scratch/bac/species_0012/run_orca.sh
sbatch run /scratch/harris.se/guassian_scratch/bac/species_0014/run_orca.sh
sbatch run /scratch/harris.se/guassian_scratch/bac/species_0015/run_orca.sh
sbatch run /scratch/harris.se/guassian_scratch/bac/species_0016/run_orca.sh
sbatch run /

In [7]:
incomplete = []
for i in range(len(database.reference_sets['main'])):
    if i in [104, 115]:
        continue
    sp_dir = os.path.join(working_dir, f'species_{i:04}')
    ol_file = os.path.join(sp_dir, 'conformer.out')
    if not os.path.exists(ol_file):
        print(f'sbatch run {sp_dir}/run_orca.sh')
        incomplete.append(i)
        continue
    try:
        o_ess = arkane.ess.ess_factory(ol_file)
    except arkane.exceptions.LogError:
        print(f'sbatch run {sp_dir}/run_orca.sh')
        incomplete.append(i)

sbatch run /scratch/harris.se/guassian_scratch/bac/species_0107/run_orca.sh
sbatch run /scratch/harris.se/guassian_scratch/bac/species_0108/run_orca.sh
sbatch run /scratch/harris.se/guassian_scratch/bac/species_0111/run_orca.sh
sbatch run /scratch/harris.se/guassian_scratch/bac/species_0127/run_orca.sh
sbatch run /scratch/harris.se/guassian_scratch/bac/species_0135/run_orca.sh
sbatch run /scratch/harris.se/guassian_scratch/bac/species_0147/run_orca.sh
sbatch run /scratch/harris.se/guassian_scratch/bac/species_0167/run_orca.sh
sbatch run /scratch/harris.se/guassian_scratch/bac/species_0184/run_orca.sh
sbatch run /scratch/harris.se/guassian_scratch/bac/species_0197/run_orca.sh
sbatch run /scratch/harris.se/guassian_scratch/bac/species_0254/run_orca.sh
sbatch run /scratch/harris.se/guassian_scratch/bac/species_0255/run_orca.sh
sbatch run /scratch/harris.se/guassian_scratch/bac/species_0256/run_orca.sh
sbatch run /scratch/harris.se/guassian_scratch/bac/species_0257/run_orca.sh
sbatch run /

In [8]:
len(incomplete)

28

# delete intermediate files

In [13]:


# for i in incomplete:
    
#     cf_files = glob.glob(os.path.join(working_dir, f'species_{i:04}', 'conformer.*'))
# #     print(cf_files)
#     for j in cf_files:
#         os.remove(j)


# redo failed runs in series instead of using parallelism

In [14]:
# for i in range(len(database.reference_sets['main'])):
for i in incomplete:
#     try:
#         sp = database.get_species_from_index(i)
#     except ValueError:
#         continue
        
    sp_dir = os.path.join(working_dir, f'species_{i:04}')
    
    gaussian_log = os.path.join(sp_dir, 'sp.log')
    # make an orca file
    
    try:
        my_log = arkane.ess.ess_factory(gaussian_log)
    except arkane.exceptions.LogError:
        continue
    # make a run file
    coord, number, mass = my_log.load_geometry()
    my_atoms = ase.Atoms(number, coord)
    
    
    parallel = True
    orca_input_file = os.path.join(sp_dir, 'conformer.inp')
    input_format = """!{res}HF {level_of_theory} TightSCF tightPNO
!energy

%maxcore 7000
{opt_parallel_line}

* xyz {charge} {mult}
{xyz}*


"""
    charge = database.reference_sets['main'][i].charge
    multiplicity = database.reference_sets['main'][i].multiplicity
    input_content = input_format.format(
        res='r' if multiplicity == 1 else 'u',
        level_of_theory='dlpno-ccsd(t)-f12 cc-pvtz-f12 aug-cc-pvtz/c cc-pvtz-f12-cabs',
        opt_parallel_line='%pal nprocs 1 end' if parallel else '',
        charge=charge,
        mult=multiplicity,
        xyz=get_xyz(my_atoms),
    )
    
    with open(orca_input_file, 'w') as f:
        f.write(input_content)
        
    run_orca_script = os.path.join(sp_dir, 'run_orca.sh')
    with open(run_orca_script, 'w') as f:
        # TODO format the text without """ so it doesn't mess with VSCode's collapse function button
        f.write("""#!/bin/bash
#SBATCH --job-name=""" + f'orca_{i:04}' + """
#SBATCH --error=error.log
#SBATCH --nodes=1
#SBATCH --partition=west,short
#SBATCH --exclude=c5003
#SBATCH --mem=200Gb
#SBATCH --time=24:00:00


ompi=/work/westgroup/orca/openmpi-4.1.6/build
PATH=$ompi/bin:$PATH
LD_LIBRARY_PATH=$ompi/lib:$ompi/etc:$LD_LIBRARY_PATH

#Orca
orcadir=/work/westgroup/orca/orca_6_0_1_linux_x86-64_shared_openmpi416
export PATH=$PATH:$orcadir
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$orcadir

$orcadir/orca conformer.inp > conformer.out
""")

In [12]:
ol = os.path.join(sp_dir, 'conformer.out')

In [13]:
o_ess = arkane.ess.ess_factory(ol)

In [39]:
valid_indices = []
for i in range(len(database.reference_sets['main'])):
#     try:
#         sp = database.get_species_from_index(i)
#     except ValueError:
#         continue
        
    sp_dir = os.path.join(working_dir, f'species_{i:04}')
    
    gaussian_log = os.path.join(sp_dir, 'sp.log')
    # make an orca file
    
    try:
        my_log = arkane.ess.ess_factory(gaussian_log)
    except arkane.exceptions.LogError:
        continue
    # make a run file
    coord, number, mass = my_log.load_geometry()
    valid_indices.append(i)

In [40]:
valid_indices

[0,
 1,
 3,
 8,
 9,
 10,
 13,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 26,
 27,
 30,
 31,
 32,
 34,
 35,
 37,
 38,
 41,
 47,
 48,
 49,
 54,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 

In [30]:
coord

array([[ 1.951141e+00, -7.805600e-02,  2.100000e-05],
       [ 5.317380e-01,  3.952630e-01,  4.000000e-05],
       [-5.317630e-01, -3.953270e-01, -1.400000e-05],
       [-1.951166e+00,  7.799600e-02,  8.000000e-06],
       [ 2.001327e+00, -1.166181e+00, -7.600000e-05],
       [ 2.487778e+00,  2.892070e-01,  8.767690e-01],
       [ 2.487809e+00,  2.893660e-01, -8.766420e-01],
       [ 3.786690e-01,  1.471550e+00,  1.050000e-04],
       [-3.786970e-01, -1.471614e+00, -8.000000e-05],
       [-2.487821e+00, -2.893100e-01, -8.767110e-01],
       [-2.487817e+00, -2.893810e-01,  8.767000e-01],
       [-2.001348e+00,  1.166120e+00,  5.300000e-05]])

In [33]:
my_atoms = ase.Atoms(number, coord)

In [34]:
my_atoms

Atoms(symbols='C4H8', pbc=False)

In [28]:
database.reference_sets['main'][0].charge

0

In [20]:
database.get_species_from_index(10)[0]

<ReferenceSpecies O(10)>

In [22]:
len(database.reference_sets['main'])

421

In [19]:
for i in range(len(database.reference_sets['main'])):
    if 'Br' in database.reference_sets['main'][i].smiles:
        print(i, database.reference_sets['main'][i].smiles)

51 OCCBr
118 ClBr
119 FBr
120 [O]Br
121 FC(F)Br
122 ClC(Cl)(Cl)Br
123 FC(F)(F)Br
168 BrBr
169 O=C(Br)Br
215 CCBr
253 Br
268 [O-]Br
269 OBr
290 CBr
394 C=CBr
